In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import numpy as np
from skimage import data
from skimage.transform import resize

import matplotlib.pyplot as plt

import torch

from src.reader import KineticDataset

from src.vqgan import ViTVQGAN
from src.my_model import MaskCode
from src.trainer import AttentionMaskModeling


In [3]:
path = "E:/kinetics-dataset/k400"
split = "train"

ds = KineticDataset(
    path, split,
    n_frames=32,
)
ds_loader = DataLoader(
    ds, 1, True, 
)

99/246534

In [ ]:
for batch in ds_loader:
    code, _ = batch
    break

B, T, HW = code.shape

In [ ]:
lightning_model = AttentionMaskModeling(
    model=MaskCode(
		window_size=(3, 3),
		length=128, height=32, width=32,
		depth=4, heads=12, dim=96, embed_dim=32,
		n_codes=8192,
	), 
)

In [ ]:
lightning_model = AttentionMaskModeling(
    model=MaskCode(
		window_size=(3, 3),
		length=32, height=32, width=32,
		depth=4, heads=12, dim=96, embed_dim=32,
		n_codes=8192,
	), 
)

lightning_model.cuda()
_batch = batch[0].cuda(), None
lightning_model.training_step(_batch, 0)

In [ ]:
is_dev = False

lightning_model = AttentionMaskModeling(
    model=MaskCode(
		window_size=(3, 3),
		length=32, height=32, width=32,
		depth=4, heads=8, dim=128, embed_dim=32,
		n_codes=8192,
	), 
    top_p=0.95
)
wandb_logger = WandbLogger(
	project="semcom",
)
# wandb_logger.experiment.config.update({
#     "dim": dim,
#     "depth": depth
# })
trainer = pl.Trainer(
    # training settings
    max_epochs=25,
    val_check_interval=1.0,
    accelerator="cpu" if is_dev else "gpu",
    precision="32-true" if is_dev else "16-mixed",
    accumulate_grad_batches=1,
    gradient_clip_val=1.0,
    # logging settings
    default_root_dir=f"./checkpoints",
    logger=wandb_logger,
    callbacks=[
        ModelCheckpoint(
			monitor="val_loss",
			dirpath="my/path/",
			filename="sample-mnist-{epoch:02d}-{val_loss:.2f}",
			save_top_k=3,
			mode="min",
		)
	]
    # dev setting
    fast_dev_run=is_dev,
)
trainer.fit(
    lightning_model, 
    train_dataloaders=ds_loader,
    # val_dataloaders=None,
)

c:\Users\ngoak\anaconda3\envs\semcom\lib\site-packages\pytorch_lightning\utilities\parsing.py:210: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
c:\Users\ngoak\anaconda3\envs\semcom\lib\site-packages\pytorch_lightning\loggers\wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` befo

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\ngoak\anaconda3\envs\semcom\lib\site-packages\torch\optim\lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
`Trainer.fit` stopped: `max_epochs=25` reached.
